In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')
import random as rnd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.patches as mpatches
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
%matplotlib inline
SEED=4242
rnd.seed(10)
SIZE=30000
ALPHA=0.03
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import collections
from sklearn.ensemble import RandomForestClassifier
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score,\
 accuracy_score, classification_report, make_scorer, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import TomekLinks
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from xgboost import plot_importance
# Time Series Features
# import tsfresh as tsf
from datetime import date, timedelta
from scipy import signal
from scipy.stats import variation
# Imbalance handling
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df= pd.read_csv('/content/DFORE488.zip', compression='zip', low_memory=True)

In [3]:
print(df.shape)
df.drop_duplicates(subset= 'email_address', keep='last', inplace= True)
print(df.shape)
df.set_index('email_address', inplace=True)
bdex= list(filter(lambda x: '-' in x, df.columns.tolist()))
df['gender']= df.gender.map({'M':0, 'F':1})
bdex.extend(['gender', 'birth_year'])
bf= df[bdex]
rf= df[df.columns.difference(bdex)]
print('BDEX Columns: ',bf.shape,'RFM Columns: ', rf.shape)


(75545, 663)
(75545, 663)
BDEX Columns:  (75545, 608) RFM Columns:  (75545, 54)


In [4]:
df.bdex_data_flag.value_counts()

2    69429
1     4169
3     1395
0      552
Name: bdex_data_flag, dtype: int64

In [5]:
(4169-1395-552)/69429

0.03200391767129009

In [6]:
good=df.loc[df.bdex_data_flag== 1].index.to_list()
bad= df.loc[df.bdex_data_flag.isin([0,3])].index.to_list()

In [7]:
import random
random.seed(4242)

In [8]:
bad.extend(random.sample(df.loc[df.bdex_data_flag.isin([2])].index.to_list(), int(.05*len(df.loc[df.bdex_data_flag.isin([2])].index.to_list()))))

In [9]:
y=pd.Series(1, index=good).append(pd.Series(0, index=bad))

In [10]:
y.value_counts()

0    5418
1    4169
dtype: int64

In [11]:
select= y.index

In [12]:
len(select)

9587

## RFM

In [13]:
rf['Recency']= rf[['Vend_Recency', 'WalletLoad_Recency']].min(axis=1)
rf['DaysBetween']= rf[['AvgDaysBetweenVends', 'AvgDaysBetweenWalletLoad']].min(axis=1)
rf['DoB']= rf[['Vend_DaysonBook','WalletLoad_DaysonBook']].min(axis=1)
print(rf[['Recency', 'DaysBetween', 'DoB']].isnull().sum())
# Consumers in 150-180 cohort who visit atleast once in 10 days and their last activity is maximum 5 days old
# flg1= (rf.Recency <= 5) & (rf.DaysBetween<=10) & (rf.DoB > 150) & (rf.DoB <= 180)
# flg0= (rf.DoB > 150) & (rf.DoB <= 180)
# print('Target Group: ', sum(flg1))
# print('Size of Consumer Base: ', sum(flg0))
# print('Minority Class Group: ', 100.00*sum(flg1)/sum(flg0))


Recency        0
DaysBetween    0
DoB            0
dtype: int64


## BDEX

In [14]:
train = bf.loc[select].fillna(0)

In [15]:
print(train.shape, y.shape)

(9587, 608) (9587,)


In [16]:
# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(train.values, y.values, test_size=0.3, random_state=4242, shuffle=True)

print('Training Sample- Minority Class: ', sum(y_train))
print('Total Training Sample: ', len(y_train))
print('%: ', 100*sum(y_train)/len(y_train))
print()
print('Training Sample- Minority Class: ', sum(y_test))
print('Total Training Sample: ', len(y_test))
print('%: ', 100*sum(y_test)/len(y_test))

Training Sample- Minority Class:  2942
Total Training Sample:  6710
%:  43.845007451564825

Training Sample- Minority Class:  1227
Total Training Sample:  2877
%:  42.64859228362878


In [17]:
%%time
from pandas.core.apply import ResamplerWindowApply
datasets={}
resamplers= {'Random under-sampling': RandomUnderSampler(),
             'Random over-sampling': RandomOverSampler(),
             'Tomek links under-sampling': TomekLinks(sampling_strategy='majority'),
             'SMOTE over-sampling': SMOTE(sampling_strategy='minority')}

for name, resampler in resamplers.items():
  X_retrain,y_retrain= resampler.fit_resample(X_train, y_train)
  datasets[name]=[X_retrain,y_retrain,X_test, y_test]
  print(name,X_retrain.shape,y_retrain.shape)

Random under-sampling (5884, 608) (5884,)
Random over-sampling (7536, 608) (7536,)
Tomek links under-sampling (6199, 608) (6199,)
SMOTE over-sampling (7536, 608) (7536,)
CPU times: user 2.36 s, sys: 247 ms, total: 2.61 s
Wall time: 1.63 s


In [18]:
from pandas.core.common import random_state
classifiers = {
    "LogisiticRegression": LogisticRegression(class_weight='balanced', random_state=SEED),
    "KNearest": KNeighborsClassifier(),
    # "Support Vector Classifier": SVC(class_weight='balanced', probability=True, random_state=SEED),
    "DecisionTreeClassifier": DecisionTreeClassifier(class_weight='balanced', random_state=SEED),
    "RandomForest": RandomForestClassifier(class_weight= 'balanced', random_state=SEED),
    "XGBClassifier": xgb.XGBRFClassifier(tree_method="hist", random_state=SEED)
}
scorer={'f1_scorer': make_scorer(f1_score),
        'auc_scorer': make_scorer(roc_auc_score),
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score)}

In [19]:
%%time
models={}
recs=[]
for dk, v in datasets.items():
  for key, classifier in classifiers.items():
    print(dk, key)
    clear_output(wait= True)
    classifier.fit(v[0], v[1])
    training_score = cross_validate(classifier,v[0], v[1], cv=5, scoring= scorer)
    y_hat= classifier.predict(v[2])
    rec=[dk+': '+key, dk, key]
    rec.extend(list(np.array(list(training_score.values())).mean(axis=1)[2:]))
    # # roc_auc_score(v[3], classifier.predict_proba(v[2])[:, 1])
    rec.extend([f1_score(v[3], y_hat),roc_auc_score(v[3], classifier.predict_proba(v[2])[:, 1]), precision_score(v[3], y_hat), recall_score(y_hat, v[3])])
    recs.append(rec)
    # models[dk+': '+key]= [classifier, v[0], v[1], v[2], v[4], v[5]]

CPU times: user 2min 16s, sys: 7.15 s, total: 2min 23s
Wall time: 1min 59s


In [20]:
results= pd.DataFrame.from_records(recs)
results.columns=['DataClassifier', 'Sampling Strategy', 'Classifiers', 'train_f1', 'train_AUC', 'train_precision', 'train_recall','test_f1', 'test_AUC', 'test_precision', 'test_recall']

In [21]:
results.sort_values('test_AUC', ascending= False).head()

,DataClassifier,Sampling Strategy,Classifiers,train_f1,train_AUC,train_precision,train_recall,test_f1,test_AUC,test_precision,test_recall
14,Tomek links under-sampling: XGBClassifier,Tomek links under-sampling,XGBClassifier,0.280693,0.518061,0.532522,0.192393,0.294085,0.545325,0.476277,0.476277
9,Random over-sampling: XGBClassifier,Random over-sampling,XGBClassifier,0.441187,0.514073,0.518997,0.384832,0.392102,0.542637,0.463333,0.463333
13,Tomek links under-sampling: RandomForest,Tomek links under-sampling,RandomForest,0.496846,0.536382,0.515849,0.479267,0.470357,0.538431,0.453788,0.453788
18,SMOTE over-sampling: RandomForest,SMOTE over-sampling,RandomForest,0.551101,0.582367,0.575505,0.545323,0.434075,0.533962,0.457169,0.457169
3,Random under-sampling: RandomForest,Random under-sampling,RandomForest,0.500150,0.506284,0.506070,0.494555,0.482572,0.533810,0.459779,0.459779


# Data Export

In [22]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=4242, shuffle=True)

In [23]:
y_train.to_csv('train_email.csv', index=True, index_label='Email', sep=',')

In [24]:
y_test.to_csv('test_email.csv', index=True, index_label='Email', sep=',')

In [25]:
pd.DataFrame(y_test.index.to_list()).to_csv('test_email_list.csv')

In [26]:
y_train.value_counts()

0    3768
1    2942
dtype: int64

In [ ]:
y_test.value_counts()

0    1650
1    1227
dtype: int64

In [ ]:
print('Training Sample- Minority Class: ', sum(y_train))
print('Total Training Sample: ', len(y_train))
print('%: ', 100*sum(y_train)/len(y_train))
print()
print('Training Sample- Minority Class: ', sum(y_test))
print('Total Training Sample: ', len(y_test))
print('%: ', 100*sum(y_test)/len(y_test))

Training Sample- Minority Class:  2942
Total Training Sample:  6710
%:  43.845007451564825

Training Sample- Minority Class:  1227
Total Training Sample:  2877
%:  42.64859228362878


# Bdex results